Google Colab referente a 2a questão do projeto da disciplina de Aprendizagem de Máquina ofertada no período de 2022.1 no CIn - UFPE.

Alunos:
* Acarcio Gomes (agaj)
* Luigi Luz (lfml)

# Instalação de dependências e bibliotecas

## Instalação de bibliotecas de terceiros e update de versões

In [ ]:
!pip install -U scipy
!pip install scikit-posthocs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importação das bibliotecas

In [ ]:
import random
import itertools
import time
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Metricas
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
#Processamento
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

#Funções de densidade
from scipy.stats import multivariate_normal
from scipy.stats import norm

#Seleção de modelos
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.base import BaseEstimator, ClassifierMixin

#models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KernelDensity
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from google.colab import drive

import warnings

import scipy
from scipy import stats
import scikit_posthocs as sp

drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


# Download e fusão dos datasets

In [ ]:
# Download the database
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00459/avila.zip'

# Unzip the database
!unzip -n avila.zip -d .

--2022-06-20 20:57:42--  https://archive.ics.uci.edu/ml/machine-learning-databases/00459/avila.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 584600 (571K) [application/x-httpd-php]
Saving to: ‘avila.zip’

avila.zip           100%[===================>] 570.90K  --.-KB/s    in 0.1s    

2022-06-20 20:57:43 (3.95 MB/s) - ‘avila.zip’ saved [584600/584600]

Archive:  avila.zip
   creating: ./avila/
  inflating: ./avila/avila-tr.txt    
  inflating: ./avila/avila-ts.txt    
  inflating: ./avila/avila-description.txt  


In [ ]:
# Definição das colunas do dataset
name = ['intercolumnar distance', 'upper margin', 'lower margin', 'exploitation', 'row number', 'modular ratio', 'interlinear spacing', 'weight', 'peak number', 'modular ratio/interlinear spacing', 'class']

In [ ]:
avila_tr = pd.read_csv("./avila/avila-tr.txt", delimiter=',', names= name)
avila_tr

,intercolumnar distance,upper margin,lower margin,exploitation,row number,modular ratio,interlinear spacing,weight,peak number,modular ratio/interlinear spacing,class
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,A
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,A
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,A
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,A
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,F
...,...,...,...,...,...,...,...,...,...,...,...
10425,0.080916,0.588093,0.015130,0.002250,0.261718,-0.557133,0.371178,0.932346,0.282354,-0.580141,F
10426,0.253730,-0.338346,0.352988,-1.154243,0.172340,-0.557133,0.257927,0.348428,0.032902,-0.527134,F
10427,0.229043,-0.000745,0.171611,-0.002793,0.261718,0.688613,0.295677,-1.088486,-0.590727,0.580142,A
10428,-0.301743,0.352558,0.288973,1.638181,0.261718,0.688613,0.069175,0.502761,0.625350,0.718969,E


In [ ]:
avila_ts = pd.read_csv("./avila/avila-ts.txt", delimiter=',', names= name)
avila_ts

,intercolumnar distance,upper margin,lower margin,exploitation,row number,modular ratio,interlinear spacing,weight,peak number,modular ratio/interlinear spacing,class
0,-3.498799,0.250492,0.232070,1.224178,-4.922215,1.145386,0.182426,-0.165983,-0.123005,1.087144,W
1,0.204355,-0.354049,0.320980,0.410166,-0.989576,-2.218127,0.220177,0.181844,2.090879,-2.009758,A
2,0.759828,-1.304042,-0.023991,-0.973663,-0.006417,-0.349509,-0.421580,-0.450127,0.469443,0.060952,I
3,-0.005490,0.360409,0.281860,-0.213479,-1.168333,-1.013906,-0.346080,1.176165,0.968347,-0.627999,E
4,0.080916,0.101320,0.104040,0.140490,0.261718,0.480988,0.710932,-0.253430,-0.497183,0.155681,A
...,...,...,...,...,...,...,...,...,...,...,...
10432,-0.128929,-0.040001,0.057807,0.557894,0.261718,-0.930856,-0.044076,1.158458,2.277968,-0.699884,X
10433,0.266074,0.556689,-0.020434,0.176624,0.261718,-0.515608,0.597681,0.178349,0.625350,-0.657245,G
10434,-0.054866,0.580242,0.032912,-0.016668,0.261718,1.519109,0.371178,-0.985508,-0.403638,1.276301,A
10435,0.080916,0.588093,0.015130,0.002250,0.261718,-0.930856,-0.270579,0.163807,-0.091823,-0.593329,F


In [ ]:
avila_df = pd.concat((avila_tr, avila_ts), ignore_index=True)
avila_df

,intercolumnar distance,upper margin,lower margin,exploitation,row number,modular ratio,interlinear spacing,weight,peak number,modular ratio/interlinear spacing,class
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,A
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,A
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,A
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,A
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,F
...,...,...,...,...,...,...,...,...,...,...,...
20862,-0.128929,-0.040001,0.057807,0.557894,0.261718,-0.930856,-0.044076,1.158458,2.277968,-0.699884,X
20863,0.266074,0.556689,-0.020434,0.176624,0.261718,-0.515608,0.597681,0.178349,0.625350,-0.657245,G
20864,-0.054866,0.580242,0.032912,-0.016668,0.261718,1.519109,0.371178,-0.985508,-0.403638,1.276301,A
20865,0.080916,0.588093,0.015130,0.002250,0.261718,-0.930856,-0.270579,0.163807,-0.091823,-0.593329,F


# Análise exploratória

In [ ]:
avila_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20867 entries, 0 to 20866
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   intercolumnar distance             20867 non-null  float64
 1   upper margin                       20867 non-null  float64
 2   lower margin                       20867 non-null  float64
 3   exploitation                       20867 non-null  float64
 4   row number                         20867 non-null  float64
 5   modular ratio                      20867 non-null  float64
 6   interlinear spacing                20867 non-null  float64
 7   weight                             20867 non-null  float64
 8   peak number                        20867 non-null  float64
 9   modular ratio/interlinear spacing  20867 non-null  float64
 10  class                              20867 non-null  object 
dtypes: float64(10), object(1)
memory usage: 1.8+ MB


In [ ]:
avila_df= avila_df.dropna(axis='rows')

classes = np.array(pd.unique(avila_df[avila_df.columns[-1]]), dtype=str)
nrow, ncol = avila_df.shape
print("Matriz de atributos: Número de linhas: ", nrow," colunas: ", ncol)
attributes = list(avila_df.columns)
avila_df.head(10)

Matriz de atributos: Número de linhas:  20867  colunas:  11


,intercolumnar distance,upper margin,lower margin,exploitation,row number,modular ratio,interlinear spacing,weight,peak number,modular ratio/interlinear spacing,class
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,A
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,A
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,A
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,A
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,F
5,0.117948,-0.220579,-3.210528,-1.623238,0.261718,-0.349509,0.257927,-0.385979,-0.247731,-0.331310,A
6,0.389513,-0.220579,-3.210528,-2.624155,0.261718,-0.764757,0.484429,-0.597510,-0.372457,-0.810261,A
7,0.019197,-0.040001,0.288973,-0.042597,0.261718,-1.013906,0.069175,0.890701,0.095265,-0.842014,F
8,0.500607,0.140576,0.388552,-0.637358,0.261718,-0.681707,0.295677,0.931046,0.500624,-0.642297,H
9,-0.252367,0.069915,0.246296,0.523550,0.261718,-1.221530,0.899684,1.373076,0.625350,-1.400890,E


In [ ]:
avila_df['class'].value_counts()

A    8572
F    3923
E    2190
I    1663
X    1044
H    1039
G     893
D     705
Y     533
C     206
W      89
B      10
Name: class, dtype: int64

É possível observar que a base de dados é desbalanceada em relação a alguma das classes, de modo que a classe mais frequente, "A", possui 2 vezes mais amostras que a segunda mais frequente "F" e aproximadamente 850 vezes mais amostras que a classe menos frequente, "B".

# Separação dos dados em treino, validação e teste

Quando necessário, retire do conjunto de treinamento, um conjunto de validação (20%) para fazer ajuste de hiper-parametros e depois treine o modelo novamente com o conjunto de aprendizagem + validação. Use amostragem estratificada.

In [ ]:
# Separação dos atributos e da classe
X = avila_df.drop(columns=["class"]).to_numpy()

le = LabelEncoder()
# Convertemos as labels em valores numericos
y = le.fit_transform(avila_df["class"])

In [ ]:
# Repeated Stratified KFold
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html

rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=30, random_state=1)

In [ ]:
p = 0.8 # Proporção do dataset de treino
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state=42, stratify=y)

In [ ]:
# Padronização dos dados de treinamento
scaler = StandardScaler()

# Fazemos o fit apenas no conjunto de treino
scaler.fit(X_train) 

# Fazemos o transform no conjunto de treino e de teste
x_train = scaler.transform(X_train)
x_test = scaler.transform(X_test)
X = scaler.transform(X)

# Observamos a média e o desvio padrão dos dados transformados
print("Dados transformados: ")
print("Media: ", np.mean(X_train, axis=0))
print("Desvio Padrão: ", np.std(X_train, axis=0))
# Com a padronização, o esperado é que os dados apresentem média nula e desvio padrão unitário

Dados transformados: 
Media:  [-0.00338592  0.00199001  0.00320414  0.00281082  0.0056412   0.00272913
 -0.00180075 -0.0054599  -0.00109031  0.00074361]
Desvio Padrão:  [0.96635671 0.97204001 0.98649371 0.9946932  0.9923333  0.99757281
 1.00578092 0.9965013  1.0048708  0.9997608 ]


# Funções auxiliares

In [ ]:
def __evaluate_model(model, model_name):
  drive_root_path = "/content/drive/My Drive"
  scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_macro': 'recall_macro',
           'f1_macro': 'f1_macro'}
  df_columns= ["fit_time", "score_time","test_acc","test_f1_macro", "test_prec_macro", "test_rec_macro"]
  warnings.filterwarnings('ignore')
  scores = cross_validate(model, X, y, scoring=scoring, cv=rskf, error_score='raise', n_jobs=-1)
  
  scores_df = pd.DataFrame(scores, columns= df_columns)
  scores_df.to_csv(f"{drive_root_path}/{model}.csv")

  #print (scores)
  mean_metric_acc = np.round(np.mean(1-scores["test_acc"]), 2)
  std_metric_acc = np.round(np.std(1-scores["test_acc"]), 2)
  mean_metric_f1 = np.round(np.mean(scores["test_f1_macro"]), 2)
  std_metric_f1 = np.round(np.std(scores["test_f1_macro"]), 2)
  mean_metric_prec = np.round(np.mean(scores["test_prec_macro"]), 2)
  std_metric_prec = np.round(np.std(scores["test_prec_macro"]), 2)
  mean_metric_rec = np.round(np.mean(scores["test_rec_macro"]), 2)
  std_metric_rec = np.round(np.std(scores["test_rec_macro"]), 2)

  print(f"Modelo {model_name} com taxa de erro média de {mean_metric_acc} (+- {std_metric_acc})")
  print(f"Modelo {model_name} com f1 médio de {mean_metric_f1} (+- {std_metric_f1})")
  print(f"Modelo {model_name} com precisão média de {mean_metric_prec} (+- {std_metric_prec})")
  print(f"Modelo {model_name} com recall médio de {mean_metric_rec} (+- {std_metric_rec})")

  return scores

# Treinamento dos classificadores

## (i) Classificador bayesiano gaussiano

In [ ]:
# Classe que implementa o classificador bayesiano gaussiano
class bayesianoGaussiano(BaseEstimator, ClassifierMixin):
    def __init__(self,):
        self.m = []
        self.cv =[]

    def fit(self, X, y):
      self.classes = np.unique(y)
      self.probPriori = np.zeros(len(self.classes))
      for c in np.arange(0, len(self.classes)):
          elements = tuple(np.where(y == self.classes[c]))
          self.probPriori[c]= len(elements[0])/len(y)
          Z = X[elements, :][0]
          self.m.append(np.mean(Z, axis=0))
          self.cv.append(np.cov(np.transpose(Z)))

    def predict(self, X):
      y_pred = []
      P = pd.DataFrame(data=np.zeros((X.shape[0], len(self.classes))), columns = self.classes)

      for c in np.arange(0, len(self.classes)):
          for j in np.arange(0, X.shape[0]):
              x_p = X[j, :]
              pj = multivariate_normal.pdf(x_p, mean=self.m[c], cov=self.cv[c], allow_singular=True)
              P[self.classes[c]][j] = pj*self.probPriori[c]

      for i in np.arange(0, X.shape[0]):
          Class = np.argmax(np.array(P.iloc[[i]]))
          y_pred.append(self.classes[Class])

      return np.array(y_pred)


In [ ]:
ref_bg_model = bayesianoGaussiano()
ref_bg_model_scores = __evaluate_model(ref_bg_model, "Classificador bayesiano gaussiano")

Modelo Classificador bayesiano gaussiano com taxa de erro média de 0.64 (+- 0.02)
Modelo Classificador bayesiano gaussiano com f1 médio de 0.37 (+- 0.01)
Modelo Classificador bayesiano gaussiano com precisão média de 0.37 (+- 0.01)
Modelo Classificador bayesiano gaussiano com recall médio de 0.55 (+- 0.03)


## (ii) Treine um classificador bayesiano baseado em k-vizinhos

Considere as seguintes distâncias Euclidiana, City-Block e Chebyshev para definir a vizinhança. Use conjunto de validação para fixar o número de vizinhos k e a distância.

In [ ]:
knn_base_model = KNeighborsClassifier()

knn_params_grid = {
    "n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "metric": ["euclidean", "cityblock", "chebyshev"]
}

# Cria a busca parametrica
gs = GridSearchCV(knn_base_model, knn_params_grid, cv=5, n_jobs=-1, scoring="f1_macro", return_train_score=True)
# Faz o treinamento com a busca parametrica
gs_results = gs.fit(x_train, y_train)

print(f"Os parametros do melhor modelo foram = {gs_results.best_params_}")

Os parametros do melhor modelo foram = {'metric': 'cityblock', 'n_neighbors': 1}


In [ ]:
best_knn_clf = KNeighborsClassifier(n_neighbors=1, metric="cityblock")
knn_scores = __evaluate_model(best_knn_clf, "Classificador k-vizinhos")
knn_scores

Modelo Classificador k-vizinhos com taxa de erro média de 0.12 (+- 0.01)
Modelo Classificador k-vizinhos com f1 médio de 0.89 (+- 0.01)
Modelo Classificador k-vizinhos com precisão média de 0.9 (+- 0.01)
Modelo Classificador k-vizinhos com recall médio de 0.88 (+- 0.01)


{'fit_time': array([0.04083323, 0.03911185, 0.03561926, 0.03562713, 0.03729987,
        0.03451943, 0.03671312, 0.04289699, 0.03887677, 0.03778124,
        0.03681064, 0.03868389, 0.03923202, 0.03483725, 0.03911853,
        0.0368948 , 0.03940153, 0.03922796, 0.03810692, 0.03499722,
        0.03701448, 0.03581882, 0.0381372 , 0.03485918, 0.03811669,
        0.03563595, 0.0380888 , 0.03794336, 0.0440321 , 0.03681469,
        0.03581691, 0.03585196, 0.03751993, 0.04792452, 0.03640819,
        0.03513575, 0.03634381, 0.03820801, 0.03677177, 0.03568029,
        0.04178953, 0.03609896, 0.03846407, 0.03716588, 0.04121518,
        0.03659749, 0.03812265, 0.03697157, 0.036834  , 0.03521347,
        0.04916644, 0.03702474, 0.04134107, 0.03669786, 0.04286814,
        0.05212927, 0.0374186 , 0.03721499, 0.03606677, 0.03741193,
        0.0365026 , 0.0356648 , 0.04390979, 0.03589296, 0.03655148,
        0.035146  , 0.03662968, 0.03542304, 0.03646779, 0.03546166,
        0.03919721, 0.03657103, 0.04

## (iii) Treine um classificador bayesiano baseado em janela de Parzen

Use a funçaõ de kernel multivariada produto com o mesmo h para todas as dimensões a função de kernel Gaussiana unidimensional. Use conjunto de validação para fixar o parâmetro h.

*   Equação da função discriminante:

*   Considerando a janela de parzen - método kernel:

*   Classificador:

In [ ]:
class janelaParzen(BaseEstimator, ClassifierMixin):
    def __init__(self, h=None):
        self.h = h 
        # evitar prealocacao com o z
        self.Z = []
        # evitar prealocacao com o n
        self.n = []
        self.p = 0
        
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.probPriori = np.zeros(len(self.classes))
        self.p = len(X[0])
        for c in np.arange(0, len(self.classes)):
            elements = tuple(np.where(y == self.classes[c]))
            self.probPriori[c]= len(elements[0])/len(y)
            self.Z.append(X[elements, :][0])
            self.n.append(len(X[elements, :][0]))

        return self

    def norm_numpy(self, x):
      exponential = np.power(-x, 2)
      divide_exponential = np.exp(np.divide(exponential, 2))

      return np.divide(divide_exponential, np.sqrt(2*np.pi))

    def predict(self, X):
        # provavelmente o y pred sendo alocado dinamicamente deve ser problematico
        y_pred = []
        P = pd.DataFrame(data=np.zeros((X.shape[0], len(self.classes))), columns = self.classes)

        for c in np.arange(0, len(self.classes)):
            for k in np.arange(0, X.shape[0]):
                x = X[k, :]
                somatorio=0
                for n in np.arange(0, self.n[c]):
                    produtorio=1
                    for j in np.arange(0, self.p):
                        Kj = norm.pdf((x[j]-self.Z[c][n][j])/self.h)
                        # Kj = self.norm_numpy((x[j]-self.Z[c][n][j])/self.h)
                        produtorio = produtorio*Kj
                    somatorio=somatorio + produtorio
                # a potenciacao tambem pode ser problematica
                prob=(1/(self.n[c]*math.pow(self.h,self.p)))*somatorio
                P[self.classes[c]][k] = prob*self.probPriori[c]

        for i in np.arange(0, X.shape[0]):
            Class = np.argmax(np.array(P.iloc[[i]]))
            y_pred.append(self.classes[Class])

        return np.array(y_pred)

In [ ]:
class janelaParzenVetorizado(BaseEstimator, ClassifierMixin):
    def __init__(self, h=None):
        self.h = h 
        self.Z = []
        self.n = []
        self.p = 0
        self.X_train = None
        self.y_train = None

    def __norm_numpy(self, x):
      exponential = -1 * np.power(x, 2)
      divide_exponential = np.exp(np.divide(exponential, 2))

      return np.divide(divide_exponential, np.sqrt(2*np.pi))

    def __vectorized_kernel_func(self, x):
        vect_func = np.vectorize(norm.pdf)

        return vect_func(x)
        
    def fit(self, X, y):
        # armazenamento dos dados de treinamento
        self.X_train = X
        self.y_train = y 

        # seleciona as classes do conjunto de treino
        unique, counts = np.unique(self.y_train, return_counts=True)

        self.classes = unique # pega os valores unicos do vetor y_train
        self.probPriori = counts / len(self.y_train) # pega a frequencia dos valores do vetor y_train

        self.p = len(self.classes)

        return self

    def predict(self, X):
        classes_probability = []

        # seleciona a amostra que quer ser predita
        for current_sample in range(X.shape[0]):
          
          # varia entre todas as classes possiveis
          current_sample_probabilities = np.zeros(self.p)

          for current_class_index in range(self.p):
            current_class_samples_index = np.where(self.y_train == self.classes[current_class_index])
            current_class_samples = self.X_train[current_class_samples_index]

            n_of_current_samples = current_class_samples.shape[0]

            sub = np.subtract(X[current_sample], current_class_samples) # subtraction (xi - xij)
            div = np.divide(sub, self.h) # divisao por h
            fun = self.__norm_numpy(div) # aplicaca a função de kernel
            prod = np.product(fun, axis=1) # produtorio 
            class_sum = np.sum(prod) # soma dos produtorios (talvez a bronca esteja aqui)
            class_probability_aux = (1 / (n_of_current_samples * np.power(self.h, self.p))) * class_sum
            class_probability = class_probability_aux * self.probPriori[current_class_index]

            current_sample_probabilities[current_class_index] = class_probability
          
          classes_probability.append(current_sample_probabilities)
        
        y_pred = self.classes[np.argmax(np.array(classes_probability), axis=1)]

        return y_pred

In [ ]:
parzen_base_model = janelaParzenVetorizado()

parzen_params_grid = {
    "h": [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2, 5]
}

start_time = time.time()

# Cria a busca parametrica
# Escolhemos cv igual a 5 para garantir que o conjunto de validação tem 20% do conjunto de treinamento
gs_parzen = GridSearchCV(parzen_base_model, parzen_params_grid, cv=5, n_jobs=-1, scoring='f1_macro', return_train_score=True)
# Faz o treinamento com a busca parametrica
gs_parzen_results = gs_parzen.fit(x_train, y_train)

end_time = time.time()
print(f"Elapsed time for hyperparameter search = {end_time - start_time}")

print(f"Os parametros do melhor modelo foram = {gs_parzen_results.best_params_}")

Elapsed time for hyperparameter search = 3542.6084949970245
Os parametros do melhor modelo foram = {'h': 0.1}


In [ ]:
start_time = time.time()

ref_parzen_window_model = janelaParzenVetorizado(h=0.1)
ref_parzen_window_model_scores = __evaluate_model(ref_parzen_window_model, "h_01Classificador Janela de Parzen")

end_time = time.time()
print(f"Tempo para gerar os resultados com validacao cruzada estratificada repetida = {end_time - start_time}")

Modelo h_01Classificador Janela de Parzen com taxa de erro média de 0.19 (+- 0.01)
Modelo h_01Classificador Janela de Parzen com f1 médio de 0.81 (+- 0.01)
Modelo h_01Classificador Janela de Parzen com precisão média de 0.83 (+- 0.01)
Modelo h_01Classificador Janela de Parzen com recall médio de 0.8 (+- 0.02)
Tempo para gerar os resultados com validacao cruzada estratificada repetida = 3971.4128637313843


## (iv) Treine um classificador baseado em regressão logística 

Treinar um classificador baseado em regressão logística para cada classe e use a abordagem "um contra todos" para classificar os exemplos

In [ ]:
lr = LogisticRegression(multi_class='ovr', random_state=1)
lr_scores = __evaluate_model(lr, "Classificador regressão logistica")
lr_scores

Modelo Classificador regressão logistica com taxa de erro média de 0.47 (+- 0.0)
Modelo Classificador regressão logistica com f1 médio de 0.34 (+- 0.01)
Modelo Classificador regressão logistica com precisão média de 0.41 (+- 0.02)
Modelo Classificador regressão logistica com recall médio de 0.35 (+- 0.01)


{'fit_time': array([1.69622254, 1.77094173, 1.45673227, 1.57570934, 1.11083126,
        1.06479692, 1.34476781, 1.60333395, 2.0461781 , 1.5245471 ,
        1.46574211, 1.47905469, 1.00430512, 1.02430844, 1.04982686,
        0.7813859 , 1.10187459, 0.98610735, 1.01929164, 0.92692494,
        1.05086517, 1.04157734, 0.98217034, 1.02618146, 1.12195444,
        1.0474782 , 0.76438785, 1.08801746, 1.0822823 , 0.98094606,
        0.94518781, 0.93382168, 1.08719468, 1.07167888, 1.10345197,
        1.08344793, 0.76784897, 1.02092171, 0.95489812, 1.0538609 ,
        1.0371139 , 1.01146317, 1.14148664, 1.0589757 , 1.13666463,
        0.98547244, 1.03064704, 0.76639128, 1.05507112, 0.98756766,
        1.0658679 , 1.05652332, 1.09431505, 1.05084467, 0.7540834 ,
        0.96057773, 1.07636786, 1.12217498, 1.03690147, 0.9862535 ,
        0.94243646, 1.04545689, 1.09342909, 1.07696652, 1.07325411,
        0.77814722, 1.08928013, 1.08362484, 0.9226768 , 1.06691694,
        1.124089  , 1.1057024 , 0.96

## (v) Treine um classificador usando a regra do voto majoritário à partir dos 4 classificadores bayesianos

- Classificador bayesiano gaussiano
- K-vizinhos
- Janela de parzen
- Regressão logística

In [ ]:
# Referencia classificador voto majoritário:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

clf1 = bayesianoGaussiano()
clf2 = KNeighborsClassifier(n_neighbors=1, metric="cityblock")
clf3 = janelaParzenVetorizado(h=0.2)
clf4 = LogisticRegression(multi_class='ovr', random_state=1)

base_estimators = [
              ('BG', clf1),
              ('kNN', clf2),
              ('Parzen', clf3),
              ('RL', clf4)
]

# Selecionamos o parametro voting='hard' para definir o classificador com base no voto majoritario
vm_clf = VotingClassifier(estimators=base_estimators, voting='hard', n_jobs=-1)

voto_majoritario_scores = __evaluate_model(vm_clf, "Classificador voto majoritario")

# Estimativa pontual e intervalo de confiança

(b) Obtenha uma estimativa pontual e um intervalo de confiança para cada métrica de avaliação do classificador (taxa de erro, precisão, cobertura, f-measure)

In [ ]:
def confidence_interval(dataframe, model, metric, conf_level=0.95):
  data = (dataframe[metric], )
  res = stats.bootstrap(data, np.mean, confidence_level=conf_level, random_state=42)

  low = res.confidence_interval.low
  high = res.confidence_interval.high
  pontual = np.mean([low, high])

  estimativas = [model, metric, pontual, low, high, conf_level]

  return estimativas

In [ ]:
models_prefix = ["knn", "regressao_logistica", "bayesiano_gaussiano", "janela_parzen", "voto_majoritario"]
metrics = ["test_acc",	"test_f1_macro",	"test_prec_macro",	"test_rec_macro", "test_error_rate"]

estimativas_list = []

for current_prefix in models_prefix:
  current_df = pd.read_csv(f"{current_prefix}.csv")
  current_df = current_df.drop(columns=["Unnamed: 0"])
  current_df["test_error_rate"] = 1 - current_df["test_acc"]

  for current_metric in metrics:
    estimativas = confidence_interval(current_df, current_prefix, current_metric)
    estimativas_list.append(estimativas)

estimativas_df = pd.DataFrame(estimativas_list, columns=["Modelo", "Métrica", "Estimativa pontual média", "Limite inferior", "Limite superior", "Nível de confiança"])
estimativas_df

,Modelo,Métrica,Estimativa pontual média,Limite inferior,Limite superior,Nível de confiança
0,knn,test_acc,0.877298,0.876492,0.878105,0.95
1,knn,test_f1_macro,0.889764,0.888681,0.890847,0.95
2,knn,test_prec_macro,0.899765,0.898600,0.900930,0.95
3,knn,test_rec_macro,0.882231,0.880936,0.883526,0.95
4,knn,test_error_rate,0.122702,0.121895,0.123508,0.95
5,regressao_logistica,test_acc,0.533690,0.533153,0.534228,0.95
6,regressao_logistica,test_f1_macro,0.341374,0.340173,0.342575,0.95
7,regressao_logistica,test_prec_macro,0.412870,0.410774,0.414967,0.95
8,regressao_logistica,test_rec_macro,0.346863,0.346081,0.347645,0.95
9,regressao_logistica,test_error_rate,0.466310,0.465772,0.466847,0.95


In [ ]:
teste_hipotese_df = estimativas_df[["Modelo", "Métrica", "Estimativa pontual média"]]
teste_hipotese_df

,Modelo,Métrica,Estimativa pontual média
0,knn,test_acc,0.877298
1,knn,test_f1_macro,0.889764
2,knn,test_prec_macro,0.899765
3,knn,test_rec_macro,0.882231
4,knn,test_error_rate,0.122702
5,regressao_logistica,test_acc,0.533690
6,regressao_logistica,test_f1_macro,0.341374
7,regressao_logistica,test_prec_macro,0.412870
8,regressao_logistica,test_rec_macro,0.346863
9,regressao_logistica,test_error_rate,0.466310


In [ ]:
pivot_df = teste_hipotese_df.pivot_table('Estimativa pontual média', 'Métrica', 'Modelo')
pivot_df

Modelo,bayesiano_gaussiano,janela_parzen,knn,regressao_logistica,voto_majoritario
Métrica,,,,,
test_acc,0.355269,0.806206,0.877298,0.533690,0.827568
test_error_rate,0.644731,0.193794,0.122702,0.466310,0.172432
test_f1_macro,0.369694,0.814411,0.889764,0.341374,0.835874
test_prec_macro,0.372786,0.831615,0.899765,0.412870,0.883447
test_rec_macro,0.550281,0.801785,0.882231,0.346863,0.801115


In [ ]:
teste_df = pivot_df.loc[['test_acc', 'test_f1_macro', 'test_prec_macro', 'test_rec_macro'], :]
teste_df

Modelo,bayesiano_gaussiano,janela_parzen,knn,regressao_logistica,voto_majoritario
Métrica,,,,,
test_acc,0.355269,0.806206,0.877298,0.533690,0.827568
test_f1_macro,0.369694,0.814411,0.889764,0.341374,0.835874
test_prec_macro,0.372786,0.831615,0.899765,0.412870,0.883447
test_rec_macro,0.550281,0.801785,0.882231,0.346863,0.801115


# Testes de hipótese 

(c) Usar o Friedman test (teste não parametrico) para comparar os classificadores, e o pós teste (Nemenyi test)

## Friedman test

In [ ]:
statistic, pvalue = stats.friedmanchisquare(teste_df['bayesiano_gaussiano'].values, teste_df['janela_parzen'].values, teste_df['knn'].values, teste_df['regressao_logistica'].values, teste_df['voto_majoritario'].values)

print("p valor = ", pvalue)
if (pvalue < 0.05): # alpha = 0.05
  print("Hipotese nula rejeitada, os dados vem de distribuições diferentes")
else:
  print("Hipotese nula não rejeitada, os dados pertencem a mesma distribuição")
print("\n\n")

# Se a hipotese nula for rejeitada, podemos seguir para o pós teste
# Se a hipotese nula não for rejeitada, paramos por aqui

p valor =  0.005606971834108885
Hipotese nula rejeitada, os dados vem de distribuições diferentes





In [ ]:
statistic

14.600000000000009

## Nemenyi test

In [ ]:
# Referencia para o teste de nemenyi: https://www.statology.org/nemenyi-test-python/
# A função devolve o p valor entre os pares

print(sp.posthoc_nemenyi_friedman(teste_df.to_numpy()))
print("\n\n")

          0         1         2         3         4
0  1.000000  0.517029  0.015044  0.900000  0.259856
1  0.517029  1.000000  0.517029  0.517029  0.900000
2  0.015044  0.517029  1.000000  0.015044  0.770836
3  0.900000  0.517029  0.015044  1.000000  0.259856
4  0.259856  0.900000  0.770836  0.259856  1.000000



